In [1]:
import torch
import torch.nn as nn
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob


from tqdm import tqdm

from utils import visualize, plot_data
from scipy.io import loadmat

#for 01-D2CNet added by DTY
import torch.nn.init as init
from torchvision import models
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import math

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [2]:
path = '/mnt/home/zpengac/USERDIR/count/drone_dataset'
#train_images = path + '/images'
#test_images = path + '/test_images/images'
#anno = path + '/annotation'
images = path + '/rf_image_people'
density_maps = path + '/rf_GT_people'

LOG_PARA = 1000

In [3]:
def get_train_transforms():
    return A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            #A.Resize(360,640,interpolation=2),
            #A.RandomSizedCrop(min_max_height=(409, 512), height=409, width=512, p=1.0),
            #A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=1.0),
        ],
        #additional_targets={'image': 'image','image1': 'image'}
        #keypoint_params = A.KeypointParams(format='xy')
)

def get_train_image_only_transforms():
    return A.Compose(
        [
            #A.Resize(360,640),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.Blur(blur_limit=3,p=0.2),
            A.Normalize(mean=mean,std=std,p=1.0,max_pixel_value=1.0),
            ToTensorV2(p=1.0),
        ],
        additional_targets={'image': 'image'}
    )

def get_valid_trainsforms():
    return A.Compose(
        [
            #A.Resize(360,640,interpolation=2),
            A.Normalize(mean=mean,std=std,p=1.0,max_pixel_value=1.0),
            ToTensorV2(p=1.0),
        ]
    )

# def get_valid_image_only_transforms():
#     return A.Compose(
#         [
#             A.Resize(360,640),
#         ],
#         additional_targets={'image': 'image'}
#     )

mean = torch.tensor([0.4939, 0.4794, 0.4583])
std = torch.tensor([0.2177, 0.2134, 0.2144])

def denormalize(img):
    img = img * std[...,None,None] + mean[...,None,None]
    img = img.permute(1,2,0).cpu().numpy()
    return img

In [4]:
class Counting_Dataset(Dataset):
    def __init__(self,path,image_fnames,dmap_folder,gt_folder=None,transforms=None,mosaic=False,downsample=4):
        '''
            path: root path 
            image_fnames: path of images
            dmap_folder: density map folder, eg: /dmap
            gt_folder: gt folder, currently set to visdrone xml format, modify _get_gt_data() if needed
            transforms: iteratable, can be tuple / list ... etc
            mosaic: mix up image and density map to form a new image, set to false by default
            downsample: resize dmap
        '''
        super().__init__()
        self.path = path
        self.image_fnames = image_fnames
        self.dmap_folder = path + dmap_folder
        self.transforms = transforms
        self.mosaic = mosaic
        self.downsample = downsample
        self.gt_folder = gt_folder # test purpose
        
    def __len__(self):
        return len(self.image_fnames)
    
    def __getitem__(self,idx):
        image_id = self.image_fnames[idx]
        
        if self.mosaic and random.randint(0,1) < 0.5:
            image, density_map, gt_points = self._load_mosaic_image_and_density_map(idx)
        else:
            image, density_map, gt_points = self._load_image_and_density_map(idx)
        
        h,w = image.shape[0]//self.downsample, image.shape[1]//self.downsample
        image = cv2.resize(image,(w, h))
        density_map = cv2.resize(density_map,(w//(self.downsample*2),h//(self.downsample*2)))#,interpolation=cv2.INTER_NEAREST)
        
        # Warning: doesn't work for cutout, uncommet transform and make fix code to enable cutout
        # Reason: cutout doesn't apply to mask, so mask must be image. check 01a bottom for code
        if self.transforms:
            for tfms in self.transforms:
                aug = tfms(**{
                    'image': image,
                    'mask': density_map,
                    #'keypoints': gt_points
                })
                #image, density_map, gt_points = aug['image'], aug['mask'], aug['keypoints']
                image, density_map = aug['image'], aug['mask'] # issue with previous keypoints (albumentation?)
        
        
        return image, density_map, image_id, gt_points
        
    
    def _get_dmap_name(self,fn):
        mask_name = fn.split('/')[-1].split('.')[0]
        mask_path = self.dmap_folder + '/' + mask_name + '.npy'
        return mask_path
    
    def _load_image_and_density_map(self,idx):
        image_fname = self.image_fnames[idx]
        dmap_fname = self._get_dmap_name(image_fname)
        image = cv2.imread(image_fname)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB).astype(np.float32)
        image = image/255.
        d_map = np.load(dmap_fname,allow_pickle=True)
        
        #sanity check gt
        _, points = self._get_gt_data(idx)
        # end sanity check
        
        return image, d_map, points
    
    def _load_mosaic_image_and_density_map(self,idx):
        image_1, dmap_1, points_1 = self._load_image_and_density_map(idx)
        while True:
            idx_2 = random.randint(0,len(self.image_fnames)-1)
            if idx != idx_2:
                break
        image_2, dmap_2, points_2 = self._load_image_and_density_map(idx_2)
        
        imsize = min(*image_1.shape[:2])
        xc,yc = [int(random.uniform(imsize*0.4,imsize*0.6)) for _ in range(2)]
        h,w = image_1.shape[0], image_1.shape[1]

        pos = random.randint(0,1)
        if pos == 0: #top left
            x1a,y1a,x2a,y2a = 0,0,xc,yc # img_1
            x1b,y1b,x2b,y2b = w-xc,h-yc,w,h # img_2
        elif pos == 1: # top right
            x1a,y1a,x2a,y2a = w-xc,0,w,yc
            x1b,y1b,x2b,y2b = 0,h-yc,xc,h
        elif pos == 2: # bottom left
            x1a,y1a,x2a,y2a = 0,h-yc,xc,h
            x1b,y1b,x2b,y2b = w-xc,0,w,yc
        elif pos == 3: # bottom right
            x1a,y1a,x2a,y2a = w-xc,h-yc,w,h
            x1b,y1b,x2b,y2b = 0,0,xc,yc
        
        new_image = image_1.copy()
        new_dmap = dmap_1.copy()
        new_image[y1a:y2a,x1a:x2a] = image_2[y1b:y2b,x1b:x2b]
        new_dmap[y1a:y2a,x1a:x2a] = dmap_2[y1b:y2b,x1b:x2b]
        
        #TODO: sanity check to see generate gt
        
        new_gt_points = self._get_mixed_gt_points(points_1,points_2,(x1a,y1a,x2a,y2a),(x1b,y1b,x2b,y2b),(h,w))
        
        return new_image, new_dmap, new_gt_points
    
    '''
    The follow section blocks are for sanity check 
    to compare dmap.sum() with gt points
    remove if needed
    '''
    def _get_mixed_gt_points(self,points_1,points_2,img_1_loc, img_2_loc,img_shape):
#         fn_1, points_1 = self._get_gt_data(idx_1)
#         fn_2, points_2 = self._get_gt_data(idx_2)
        x1a,y1a,x2a,y2a = img_1_loc
        x1b,y1b,x2b,y2b = img_2_loc
        h,w = img_shape
        
        result_boxes = []
        result_boxes.append(points_2)
        result_boxes = np.concatenate(result_boxes,0)
        padw = x1a-x1b
        pady = y1a-y1b

        result_boxes[:,0] += padw
        result_boxes[:,1] += pady

        np.clip(result_boxes[:,0],0,w,out=result_boxes[:,0])
        np.clip(result_boxes[:,1],0,h,out=result_boxes[:,1])
        result_boxes = result_boxes.astype(np.int32)

        result_boxes = result_boxes[np.where(result_boxes[:,0] * result_boxes[:,1] > 0)]
        result_boxes = result_boxes[np.where(result_boxes[:,0] < w)]
        result_boxes = result_boxes[np.where(result_boxes[:,1] < h)]
        
        boxes = []
        for (x,y) in points_1:
            if x >= x1a and x <= x2a and y >= y1a and y <= y2a:
                continue
            else:
                boxes.append((x,y))
        if len(boxes) == 0:
            return result_boxes
        return np.concatenate((boxes, result_boxes),axis=0)
    
    def _get_gt_data(self,idx):
        if not self.gt_folder:
            return (None,0)
        fn = self.image_fnames[idx]
        anno_path = self.path + self.gt_folder + '/' + fn.split('/')[-1].split('.')[0] + '.mat'
        test_data = loadmat(anno_path)
        points = test_data['annotation'].astype(int)
        return fn, points

In [5]:
# ADD LOG_PARA to density map

class Crop_Dataset(Counting_Dataset):
    def __init__(self,path,image_fnames,dmap_folder,gt_folder=None,transforms=None,mosaic=False,downsample=4,crop_size=512,method='train'):
        super().__init__(path,image_fnames,dmap_folder,gt_folder,transforms,mosaic,downsample)
        self.crop_size = crop_size
        if method not in ['train','valid']:
            raise Exception('Not Implement')
        self.method = method
    
    def __getitem__(self,idx):
        fn = self.image_fnames[idx]
        
        image,density_map,gt_points = self._load_image_and_density_map(idx)
        h,w = image.shape[0], image.shape[1]
        #image = cv2.resize(image,(w, h))
        
        
        if self.method == 'train':
            #h,w = image.shape[:2]
            i,j = self._random_crop(h,w,self.crop_size,self.crop_size)
            image = image[i:i+self.crop_size,j:j+self.crop_size]
            density_map = density_map[i:i+self.crop_size,j:j+self.crop_size]
            
            gt_points = gt_points - [j,i]
            mask = (gt_points[:,0] >=0 ) * (gt_points[:,0] <= self.crop_size) * (gt_points[:,1]>=0) * (gt_points[:,1]<=self.crop_size)
            gt_points = gt_points[mask]
            density_map = cv2.resize(density_map,(self.crop_size//self.downsample,self.crop_size//self.downsample))
            
        else:
            density_map = cv2.resize(density_map,(w//self.downsample,h//self.downsample))#,interpolation=cv2.INTER_NEAREST)
            density_map = density_map[1:-1,:]
        
        if self.transforms:
            for tfms in self.transforms:
                aug = tfms(**{
                    'image': image,
                    'mask': density_map,
                    #'keypoints': gt_points
                })
                #image, density_map, gt_points = aug['image'], aug['mask'], aug['keypoints']
                image, density_map = aug['image'], aug['mask'] # issue with previous keypoints (albumentation?)
        return image, density_map*LOG_PARA, fn, gt_points
    
    def _random_crop(self, im_h, im_w, crop_h, crop_w):
        res_h = im_h - crop_h
        res_w = im_w - crop_w
        i = random.randint(0, res_h)
        j = random.randint(0, res_w)
        return i, j

In [6]:
fp = glob(images + '/*.jpg')

In [7]:
split = int(len(fp) * 0.8)
train_fp[0:split][:10]

['/mnt/home/hheat/USERDIR/counting-bench/data/images/11_233.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/12_240.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/08_113.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/03_319.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/06_176.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/05_105.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/11_204.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/14_253.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/14_129.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/20_248.jpg']

In [31]:
train_dataset = Crop_Dataset(path=path,
                             image_fnames=fp[:split],dmap_folder='/rf_GT_people',
                             #gt_folder='/annotation',
                             transforms=[get_train_transforms(),get_train_image_only_transforms()],
                             downsample=32,
                             crop_size=384
                                )

valid_dataset = Crop_Dataset(path=path,
                             image_fnames=fp[split:],dmap_folder='/rf_GT_people',
                             #gt_folder='/annotation',
                             transforms=[get_valid_trainsforms()],
                             method='valid',
                             downsample=32
                             #crop_size=448
                                )

In [32]:
class TrainGlobalConfig:
    num_workers = 4
    batch_size = 4
    n_epochs = 120 
    lr = 0.0002

    #change folder's name by DTY
    folder = 'D2CNet-VGG-drone_people-8.22-784'
    
    downsample = 32

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = True  # do scheduler.step after optimizer.step
    validation_scheduler = False  # do scheduler.step after validation stage loss

    SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
    scheduler_params = dict(
        max_lr=1e-4,
        #total_steps = len(train_dataset) // 4 * n_epochs, # gradient accumulation
        epochs=n_epochs,
        steps_per_epoch=int(len(train_dataset) / batch_size),
        pct_start=0.2,
        anneal_strategy='cos', 
        final_div_factor=10**5
    )
    
#     SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
#     scheduler_params = dict(
#         mode='min',
#         factor=0.5,
#         patience=1,
#         verbose=False, 
#         threshold=0.0001,
#         threshold_mode='abs',
#         cooldown=0, 
#         min_lr=1e-8,
#         eps=1e-08
#     )

In [33]:
class VGG16_Unet(nn.Module):
    
    def __init__(self):
        super(VGG16_Unet, self).__init__()
        
        
        vggfeat = list(models.vgg16(pretrained = False).features)
        self.feature1 = nn.Sequential(*vggfeat[0:4])
        self.feature2 = nn.Sequential(*vggfeat[4:9])
        self.feature3 = nn.Sequential(*vggfeat[9:16])
        self.feature4 = nn.Sequential(*vggfeat[16:23])
        self.feature5 = nn.Sequential(*vggfeat[23:30])
        del vggfeat
        
        self.decode1 = nn.Sequential(Conv2d(128, 64, kernel_size = 3, padding = 1))
        self.decode2 = nn.Sequential(Conv2d(256, 128, kernel_size = 3, padding = 1),
                                      Conv2d(128, 64, kernel_size = 3, padding = 1))
        self.decode3 = nn.Sequential(Conv2d(512, 256, kernel_size = 3, padding = 1),
                                      Conv2d(256,128, kernel_size = 3, padding = 1))
        self.decode4 = nn.Sequential(Conv2d(1024, 512, kernel_size = 3, padding = 1),
                                      Conv2d(512, 256, kernel_size = 3, padding = 1))
                                
        self.prob_conv = nn.Sequential(Conv2d(64, 1, kernel_size = 3, padding = 1))  
        
        self.initialize_weights()
        mod = models.vgg16(pretrained = True)

        len1 = len(self.feature1.state_dict().items())
        len2 = len1 + len(self.feature2.state_dict().items())
        len3 = len2 + len(self.feature3.state_dict().items())
        len4 = len3 + len(self.feature4.state_dict().items())
                                
        for i in range(len(self.feature1.state_dict().items())):
            list(self.feature1.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i][1].data[:]
        for i in range(len(self.feature2.state_dict().items())):
            list(self.feature2.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i+len1][1].data[:]
        for i in range(len(self.feature3.state_dict().items())):
            list(self.feature3.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i+len2][1].data[:]
        for i in range(len(self.feature4.state_dict().items())):
            list(self.feature4.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i+len3][1].data[:]
        for i in range(len(self.feature5.state_dict().items())):
            list(self.feature5.state_dict().items())[i][1].data[:] = list(mod.state_dict().items())[i+len4][1].data[:]
            
        del mod        

    @torch.cuda.amp.autocast()    
    def forward(self, im_data):

        feature1 = self.feature1(im_data)
        feature2 = self.feature2(feature1)
        feature3 = self.feature3(feature2)
        feature4 = self.feature4(feature3)
        feature5 = self.feature5(feature4)        
        
        up_feature5 = nn.functional.interpolate(feature5, scale_factor = 2, mode = "bilinear", align_corners = True)
        cat_feature4 = torch.cat((feature4, up_feature5), 1)
        de_feature4 = self.decode4(cat_feature4)
        del feature5, up_feature5, feature4, cat_feature4
        
        up_feature4 = nn.functional.interpolate(de_feature4, scale_factor = 2, mode = "bilinear", align_corners = True)
        cat_feature3 = torch.cat((feature3, up_feature4), 1)
        de_feature3 = self.decode3(cat_feature3)
        del de_feature4, up_feature4, feature3, cat_feature3
        
        up_feature3 = nn.functional.interpolate(de_feature3, scale_factor = 2, mode = "bilinear", align_corners = True)
        cat_feature2 = torch.cat((feature2, up_feature3), 1)
        de_feature2 = self.decode2(cat_feature2)
        del de_feature3, up_feature3, feature2, cat_feature2
        
        up_feature2 = nn.functional.interpolate(de_feature2, scale_factor = 2, mode = "bilinear", align_corners = True)
        cat_feature1 = torch.cat((feature1, up_feature2), 1)
        de_feature1 = self.decode1(cat_feature1)        
        del de_feature2, up_feature2, feature1, cat_feature1
            
        prob_map = self.prob_conv(de_feature1)
        
        #prob_map = torch.clamp(prob_map,0,1)
        
        return prob_map

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
                
class Conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, stride=1, relu=True, bn=False):
        super(Conv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0, affine=True) if bn else None
        self.relu = nn.ReLU(inplace=True) if relu else None

    @torch.cuda.amp.autocast()
    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x
        

In [34]:
def Gauss_initialize_weights(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.normal_(m.weight, std=0.01)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

class VGG16_frontend(nn.Module):
    def __init__(self,block_num=5,decode_num=4,load_weights=True,bn=False,IF_freeze_bn=False):
        super(VGG16_frontend,self).__init__()
        self.block_num = block_num
        self.load_weights = load_weights
        self.bn = bn
        self.IF_freeze_bn = IF_freeze_bn
        self.decode_num = decode_num

        block_dict = [[64, 64], ['M',128, 128], ['M',256, 256, 256],\
             ['M',512, 512, 512], ['M',512, 512, 512,'M']]

        self.frontend_feat = []
        for i in range(block_num):
            self.frontend_feat += block_dict[i]

        if self.bn:
            self.features = make_layers(self.frontend_feat, in_channels = 3, batch_norm=True)
        else:
            self.features = make_layers(self.frontend_feat, in_channels = 3, batch_norm=False)


        if self.load_weights:
            if self.bn:
                pretrained_model = models.vgg16_bn(pretrained = True)
            else:
                pretrained_model = models.vgg16(pretrained = True)
            pretrained_dict = pretrained_model.state_dict()
            model_dict = self.state_dict()
            # filter out unnecessary keys
            pretrained_dict_1 = {k: v for k, v in pretrained_dict.items() if k in model_dict and k != 'features.0.weight'}                    
            # overwrite entries in the existing state dict
            model_dict.update(pretrained_dict_1) 
            # load the new state dict
            self.load_state_dict(model_dict)
            weights = (list(pretrained_dict.items())[0][1].data[:,0,:,:] + \
                      list(pretrained_dict.items())[0][1].data[:,1,:,:] + \
                      list(pretrained_dict.items())[0][1].data[:,2,:,:]) / 3.
            weights = weights.unsqueeze(1)
            list(self.features.state_dict().items())[0][1].data[:] = weights

        if IF_freeze_bn:
            self.freeze_bn()
            
    @torch.cuda.amp.autocast()
    def forward(self,x):
        if self.bn: 
            x = self.features[ 0: 6](x)
            conv1_feat =x# if self.decode_num>=4 else []
            x = self.features[ 6:13](x)
            conv2_feat =x# if self.decode_num>=3 else []
            x = self.features[ 13:23](x)
            conv3_feat =x# if self.decode_num>=2 else []
            x = self.features[ 23:33](x)
            conv4_feat =x# if self.decode_num>=1 else []
            x = self.features[ 33:43+1](x)
            conv5_feat =x 
        else:
            x = self.features[ 0: 4](x)
            conv1_feat =x# if self.decode_num>=4 else []
            x = self.features[ 4:9](x)
            conv2_feat =x# if self.decode_num>=3 else []
            x = self.features[ 9:16](x)
            conv3_feat =x# if self.decode_num>=2 else []
            x = self.features[ 16:23](x)
            conv4_feat =x# if self.decode_num>=1 else []
            x = self.features[ 23:30+1](x)
            conv5_feat =x 
               
        feature_map = {'conv1':conv1_feat,'conv2': conv2_feat,\
            'conv3':conv3_feat,'conv4': conv4_feat, 'conv5': conv5_feat}   
        # feature_map = feature_map['conv5']
        return feature_map


    def freeze_bn(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()


class TasselNet_VGG16(nn.Module):
    def __init__(self,frontend_name='VGG16',block_num=5,\
        IF_pre_bn=False,IF_freeze_bn=True,load_weights=False):
        super(TasselNet_VGG16, self).__init__()

        # init parameters
        self.frontend_name = frontend_name
        self.block_num = block_num

        self.IF_pre_bn = IF_pre_bn
        self.IF_freeze_bn = IF_freeze_bn
        self.load_weights = load_weights
        self.rf = 32

        # first, make frontend
        if self.frontend_name == 'VGG16':
            self.frontend = VGG16_frontend(block_num=self.block_num,decode_num=0,\
                load_weights=self.load_weights,bn=self.IF_pre_bn,IF_freeze_bn=self.IF_freeze_bn)
             
        # --predict prob_map with conv5
        self.backend_reg = torch.nn.Sequential(
                torch.nn.Conv2d(512, 512, (1, 1) ),
                torch.nn.ReLU(),
                torch.nn.Conv2d(512, 1, (1, 1) ) ) 

        Gauss_initialize_weights(self.backend_reg) 

    @torch.cuda.amp.autocast()
    def forward(self,x):
        x = self.frontend(x)
        x = self.backend_reg(x['conv5'])
        return x

#if __name__ == '__main__':
#    mod = models.vgg16(pretrained = True)
#    #weights = torch.zeros([64, 1, 3, 3])
#    weights = torch.zeros([64, 3, 3, 3])
#    weights[:,0,:,:] = (list(mod.state_dict().items())[0][1].data[:,:,:,0] +\
#                      list(mod.state_dict().items())[0][1].data[:,:,:,1] +\
#                      list(mod.state_dict().items())[0][1].data[:,:,:,2]) / 3.
#    print(list(mod.state_dict().items())[0][1].data[:])
#    print(weights.shape)

In [27]:
img, dmap, fn, gtp = train_dataset[0]

In [28]:
m = TasselNet_VGG16()

In [29]:
pred = m(img.unsqueeze(0))

In [30]:
pred.shape

torch.Size([1, 1, 24, 24])

In [35]:
def MSELoss_MCNN(preds,targs):
    return nn.MSELoss()(preds,targs)

def MAELoss_MCNN(preds,targs,upsample):
    return nn.L1Loss()((preds/LOG_PARA).sum(dim=[-1,-2])*upsample*upsample, (targs/LOG_PARA).sum(dim=[-1,-2])*upsample*upsample)

In [36]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [43]:
import warnings
warnings.filterwarnings("ignore")

#opt_level ='O1' # apex

class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        #change path by DTY
        self.base_dir = f'/mnt/home/zpengac/USERDIR/count/drone_benchmark/{config.folder}'
        
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        
        #self.model, self.optimizer = amp.initialize(self.model,self.optimizer,opt_level=opt_level) # apex
        self.scaler = torch.cuda.amp.GradScaler()
        
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.criterion = MSELoss_MCNN
        self.metric = MAELoss_MCNN
        self.log(f'Fitter prepared. Device is {self.device}')
        
        # self.iters_to_accumulate = 4 # gradient accumulation

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss, mae_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, mse_loss: {summary_loss.avg:.8f}, time: {(time.time() - t):.5f}')
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, mae_loss: {mae_loss.avg:.8f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')

            t = time.time()
            summary_loss, mae_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, mse_loss: {summary_loss.avg:.8f}, time: {(time.time() - t):.5f}')
            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, mae_loss: {mae_loss.avg:.8f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
                    os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        mae_loss = AverageMeter()
        t = time.time()
        for step, (images, density_maps, fns, gt_pts) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'mse_loss: {summary_loss.avg:.8f}, ' + \
                        f'mae_loss: {mae_loss.avg:.8f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                batch_size = images.shape[0]
                images = images.cuda().float()
                density_maps = density_maps.cuda().float()
                

                #preds = self.model(images)
                with torch.cuda.amp.autocast(): #native fp16
                    preds = self.model(images)
                    loss = self.criterion(preds,density_maps)
                    metric_loss = self.metric(preds,density_maps,self.config.downsample)
                mae_loss.update(metric_loss.detach().item(),batch_size)
                summary_loss.update(loss.detach().item(), batch_size)
                
            #if step == 20:
            #    break

        return summary_loss, mae_loss

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        mae_loss = AverageMeter()
        t = time.time()
        for step, (images, density_maps, fns, gt_pts) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'mse_loss: {summary_loss.avg:.8f}, ' + \
                        f'mae_loss: {mae_loss.avg:.8f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            images = images.cuda().float()
            
            #Debug by DTY:
            #print("Image shape: ")
            #print(images.shape())
            #print('\n')
            #print("Debug by DTY: ------ \n")
            #print(images.type())
            #print('\n')
            #print(images.size())
            #print('\n')
            
            batch_size = images.shape[0]
            density_maps = density_maps.cuda().float()
            
            
            self.optimizer.zero_grad()
            
            with torch.cuda.amp.autocast(): #native fp16
                preds = self.model(images)
                
                #p = transforms.Compose([transforms.Scale((24*32, 24*32)), 
                #                        transforms.Pad((8,8,8,8),padding_mode='edge')])
                #preds_resized = p(preds)
                
                loss = self.criterion(preds,density_maps)
                metric_loss = self.metric(preds.detach(),density_maps.detach(),self.config.downsample)
            self.scaler.scale(loss).backward()
            
            # loss = loss / self.iters_to_accumulate # gradient accumulation
            
#             with amp.scale_loss(loss,self.optimizer) as scaled_loss: # apex
#                 scaled_loss.backward()
            #loss.backward()

            
            mae_loss.update(metric_loss.detach().item(),batch_size)
            summary_loss.update(loss.detach().item(), batch_size)
            
            #self.optimizer.step()
            self.scaler.step(self.optimizer) # native fp16
            
            if self.config.step_scheduler:
                self.scheduler.step()
            
            self.scaler.update() #native fp16
                
                
#             if (step+1) % self.iters_to_accumulate == 0: # gradient accumulation

#                 self.optimizer.step()
#                 self.optimizer.zero_grad()

#                 if self.config.step_scheduler:
#                     self.scheduler.step()
                    
            #if step == 20:
            #    break

        return summary_loss, mae_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
            #'amp': amp.state_dict() # apex
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [44]:
def collate_fn(batch):
    imgs, dmaps, fns, gt_points = zip(*batch)
    imgs = torch.stack(imgs)
    dmaps = torch.stack(dmaps).unsqueeze(1)
    return imgs,dmaps,fns,gt_points

def run_training():
    device = torch.device('cuda:0')

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=RandomSampler(train_dataset),
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
        collate_fn=collate_fn,
    )

    val_loader = torch.utils.data.DataLoader(
        valid_dataset, 
        batch_size=TrainGlobalConfig.batch_size//4,
        num_workers=TrainGlobalConfig.num_workers//2,
        shuffle=False,
        sampler=SequentialSampler(valid_dataset),
        pin_memory=True,
        collate_fn=collate_fn,
    )

    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
#     fitter.load(f'{fitter.base_dir}/last-checkpoint.bin')
    fitter.fit(train_loader, val_loader)

In [45]:
net = TasselNet_VGG16().cuda()

In [46]:
net = nn.DataParallel(net)

In [47]:
run_training()

Fitter prepared. Device is cuda:0

2021-08-22T04:49:23.837977
LR: 4.000000000000002e-06


KeyboardInterrupt: 

In [20]:
torch.cuda.device_count()

8

In [16]:
val_loader = torch.utils.data.DataLoader(
        valid_dataset, 
        batch_size=2,
        num_workers=1,
        shuffle=False,
        sampler=SequentialSampler(valid_dataset),
        pin_memory=True,
        collate_fn=collate_fn,
)

In [21]:
test_net = TasselNet_VGG16().cuda()
test_net = nn.DataParallel(test_net)

In [20]:
#change path by DTY
checkpoint = torch.load(f'/mnt/home/dty80/USERDIR/count/drone_benchmark/D2CNet-8.7-784/best-checkpoint-095epoch.bin')

test_net.load_state_dict(checkpoint['model_state_dict'])
test_net.eval()

RuntimeError: Error(s) in loading state_dict for TasselNet_VGG16:
	Missing key(s) in state_dict: "frontend.features.0.weight", "frontend.features.0.bias", "frontend.features.2.weight", "frontend.features.2.bias", "frontend.features.5.weight", "frontend.features.5.bias", "frontend.features.7.weight", "frontend.features.7.bias", "frontend.features.10.weight", "frontend.features.10.bias", "frontend.features.12.weight", "frontend.features.12.bias", "frontend.features.14.weight", "frontend.features.14.bias", "frontend.features.17.weight", "frontend.features.17.bias", "frontend.features.19.weight", "frontend.features.19.bias", "frontend.features.21.weight", "frontend.features.21.bias", "frontend.features.24.weight", "frontend.features.24.bias", "frontend.features.26.weight", "frontend.features.26.bias", "frontend.features.28.weight", "frontend.features.28.bias", "backend_reg.0.weight", "backend_reg.0.bias", "backend_reg.2.weight", "backend_reg.2.bias". 
	Unexpected key(s) in state_dict: "mod

In [53]:
imgs, dmaps, fns, points = next(iter(val_loader))

In [54]:
with torch.no_grad():
    preds = test_net(imgs.cuda())/1000

In [55]:
ps = preds.sum(dim=[-1,-2])

In [56]:
preds.shape

torch.Size([12, 1, 1528, 2720])

In [57]:
ds = dmaps.sum(dim=[-1,-2])/1000

In [65]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as pnsr

In [70]:
%%time

pre_count = []
gt_count = []
gt_points = []
avg_ssim = AverageMeter()
avg_pnsr = AverageMeter()
for step, (imgs, dmaps, fns, points) in enumerate(val_loader):
    with torch.no_grad():
        imgs = imgs.cuda().float()
        preds = test_net(imgs) / LOG_PARA
    dmaps = dmaps / LOG_PARA
    
    for pred, dmap in zip(preds, dmaps):
        pred_array = pred.detach().cpu().numpy().squeeze()
        dmap_array = dmap.detach().cpu().numpy().squeeze()
        avg_ssim.update(ssim(dmap_array, pred_array, data_range=dmap_array.max()-dmap_array.min()))
        avg_pnsr.update(pnsr(dmap_array, pred_array, data_range=dmap_array.max()-dmap_array.min()))
    
    pre_count.extend(preds.sum(dim=[-1,-2]).detach().cpu().numpy())
    
    gt_count.extend(dmaps.sum(dim=[-1,-2]).detach().cpu().numpy())
    
    gt_p = []
    for p in points:
        gt_p.append(len(p))
    gt_points.extend(gt_p)

CPU times: user 21min 44s, sys: 14min 18s, total: 36min 3s
Wall time: 33min 5s


In [71]:
mae = mean_absolute_error(pre_count,gt_count)
mse = mean_squared_error(pre_count,gt_count)
nae = mae * len(pre_count) / np.sum(gt_count)

In [73]:
def count_parameters_in_MB(model):
    return np.sum(np.prod(v.size()) for name, v in model.named_parameters() if "auxiliary" not in name) / 1e6

print(f'#Paras: {count_parameters_in_MB(test_net)}')
print(f'MAE: {mae}, MSE: {mse}, NAE: {nae}')
print(f'SSIM: {avg_ssim.avg}, PNSR: {avg_pnsr.avg}')

#Paras: 11.738809
MAE: 12.59935188293457, MSE: 597.4080810546875, NAE: 0.09619972815935798
SSIM: 0.9698961850296965, PNSR: 40.27671148514985
